In [62]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix
from sklearn.neural_network import MLPClassifier

# 1 - Modelo de Classificação Linear

In [4]:
# Inicialização do LDA
lda = LinearDiscriminantAnalysis()

In [5]:
# Carregando conjunto de treino e teste
treino = pd.read_excel('Homework03/hw3-data/cls_trated_train_set.xlsx')
teste = pd.read_excel('Homework03/hw3-data/cls_trated_test_set.xlsx')

In [6]:
# Organizando os conjuntos
treino = treino.drop(columns=['Unnamed: 0'])
teste = teste.drop(columns=['Unnamed: 0'])
x_treino = treino.T.iloc[0:-1]
y_treino = treino['MOS']
x_teste = teste.T.iloc[0:-1]
y_teste = teste['MOS']
x_treino,x_teste,y_treino,y_teste = x_treino.T,x_teste.T,y_treino.T,y_teste.T

In [7]:
lda.fit(x_treino,y_treino) # Treinando o LDA
pred = lda.predict(x_teste) # Testando o LDA
CM = confusion_matrix(y_teste,pred) # Cálculo da confusion matrix
pont = lda.score(x_teste,y_teste) # Cálculo da precisão
print(f'A análise de discriminante linear conseguiu prever {100*pont:.4}% das amostras')
print(f'\nConfusion Matrix: \n {CM}')

A análise de discriminante linear conseguiu prever 57.42% das amostras

Confusion Matrix: 
 [[1363  659    8    0   15]
 [ 182 1188    1    0    4]
 [ 163   27    8    0   18]
 [  15  104    3    0    1]
 [ 416  300   16    0   46]]


# 2 - Modelo de Classificação não Linear

## 2.1 - Análise de discriminante quadratico

In [8]:
#Inicialização do QDA
qda = QuadraticDiscriminantAnalysis()

In [9]:
qda.fit(x_treino,y_treino) # Treinando o QDA
pred_qda = qda.predict(x_teste) # Testando o QDA
CM_qda = confusion_matrix(y_teste,pred_qda) # Cálculo da confusion matrix
pont_qda = qda.score(x_teste,y_teste) # Cálculo da precisão
print(f'A análise de discriminante linear conseguiu prever {100*pont_qda:.4}% das amostras')
print(f'\nConfusion Matrix: \n {CM_qda}')

A análise de discriminante linear conseguiu prever 57.57% das amostras

Confusion Matrix: 
 [[1327  622   44   13   39]
 [ 210 1131   10    9   15]
 [ 132   25   31    0   28]
 [  20   70    7   23    3]
 [ 351  280   45    2  100]]


## 2.2 - k-Nearest Neighbors

In [59]:
#Inicialização dos KNN's com k = [1,3,5,7,9,11,13,15,17,19,21,23,25,27,29,31,35,41]
knn_ = KNeighborsClassifier(n_neighbors=1), KNeighborsClassifier(n_neighbors=3), KNeighborsClassifier(n_neighbors=5),KNeighborsClassifier(n_neighbors=7),KNeighborsClassifier(n_neighbors=9),KNeighborsClassifier(n_neighbors=11),KNeighborsClassifier(n_neighbors=13),KNeighborsClassifier(n_neighbors=15),KNeighborsClassifier(n_neighbors=17),KNeighborsClassifier(n_neighbors=19),KNeighborsClassifier(n_neighbors=21),KNeighborsClassifier(n_neighbors=23),KNeighborsClassifier(n_neighbors=25),KNeighborsClassifier(n_neighbors=27),KNeighborsClassifier(n_neighbors=29),KNeighborsClassifier(n_neighbors=31),KNeighborsClassifier(n_neighbors=35),KNeighborsClassifier(n_neighbors=41)

In [61]:
pred_knn = np.zeros([len(x_teste),len(knn_)])
CM_knn = np.zeros([len(knn_),5,5])
pont_knn = np.zeros(len(knn_))
for i in range(len(knn_)):
    knn_[i].fit(x_treino,y_treino)
    pred_knn[:,i] = knn_[i].predict(x_teste)
    CM_knn[i,:,:] = confusion_matrix(y_teste,pred_knn[:,i])
    pont_knn[i] = knn_[i].score(x_teste,y_teste)
    print(f'O K-Nearest neighbors k={knn_[i].n_neighbors} conseguiu prever {100*pont_knn[i]:.4}% das amostras')
    print(f'\nConfusion Matrix: \n {CM_knn[i]}')

O K-Nearest neighbors k=1 conseguiu prever 47.81% das amostras

Confusion Matrix: 
 [[1215.  492.   81.   30.  227.]
 [ 489.  638.   21.   31.  196.]
 [  73.   20.   42.    8.   73.]
 [  24.   41.    6.   30.   22.]
 [ 255.  196.   67.   16.  244.]]
O K-Nearest neighbors k=3 conseguiu prever 54.07% das amostras

Confusion Matrix: 
 [[1485.  417.   23.   15.  105.]
 [ 577.  704.    4.   12.   78.]
 [  96.   14.   27.    7.   72.]
 [  48.   34.    3.   26.   12.]
 [ 347.  170.   45.    5.  211.]]
O K-Nearest neighbors k=5 conseguiu prever 56.54% das amostras

Confusion Matrix: 
 [[1504.  436.   29.   12.   64.]
 [ 520.  794.    6.   11.   44.]
 [  99.   13.   24.    9.   71.]
 [  40.   36.    3.   28.   16.]
 [ 320.  195.   43.    5.  215.]]
O K-Nearest neighbors k=7 conseguiu prever 57.81% das amostras

Confusion Matrix: 
 [[1494.  442.   23.   16.   70.]
 [ 481.  838.    3.   11.   42.]
 [  98.   13.   22.    4.   79.]
 [  34.   43.    3.   29.   14.]
 [ 309.  196.   31.    2.  240.]]


## 2.3 - Support Vector Machine

In [63]:
#fazer SVM, Rede neural e talvez o SVM network